In [43]:
from aocd import get_data
from collections import defaultdict
import numpy as np
import sys

data = get_data(day=15, year=2018)

# from aocd import submit
# submit(my_answer, part="a", day=25, year=2017)

In [37]:
# data = """#######
# #G..#E#
# #E#E.E#
# #G.##.#
# #...#E#
# #...E.#
# #######"""

data = """#########
#G......#
#.E.#...#
#..##..G#
#...##..#
#...#...#
#.G...G.#
#.....G.#
#########"""

In [165]:
from IPython.display import clear_output
import time

icons = {'#': '⬛️', '.': '▫️', 'G': '👺', 'E': '🧝‍♀️'}

def print_grid(grid, round_num=None, attack_pos=None, 
               elf_hitpoints=0,
               clear=True, delay=0.003):
    if clear:
        clear_output(wait=True)
    
    print(f"Round #{round_num}\n🧝‍♀️📶: {elf_hitpoints}\n")
        
    max_x = max(x for _,x in grid.keys())
    max_y = max(y for y,_ in grid.keys())
    for x in range(max_x+1):
        for y in range(max_y+1):
            if (x,y) == attack_pos:
                print('💥', end='')
            elif grid[(x,y)][1] < 0:
                print('💀', end='')
            else:
                print(icons[grid[(x,y)][0]], end='')
        print("")

    if delay:
        time.sleep(delay)


In [167]:
dirs = [(-1,0),(0,-1),(0,1),(1,0)]
neighbours = lambda x,y: [(x+dx, y+dy) for dx,dy in dirs]


def get_dists(grid, target):
    preds = dict()

    dists = {p:0 for p,(v,_) in grid.items() if v == target}
    stack = [(p, None) for p in dists.keys()]

    while len(stack):
        pos, pred = stack.pop()
        d = dists.get(pos)

        for neigh_pos in neighbours(*pos):
            if neigh_pos != pred and grid[neigh_pos][0] == '.' and dists.get(neigh_pos, sys.maxsize) > d+1:
                dists[neigh_pos] = d+1
                stack.append((neigh_pos, pos))

    return dists

def move(grid, start, dest):
    grid[dest] = grid[start]
    grid[start] = ('.', 0)

def hit(grid, pos, hit_points=3):
    grid[pos][1] -= hit_points
    if grid[pos][1] <= 0:
        grid[pos] = ('.', -1)

In [168]:
def simulate_combat(data, elf_hitpoints=3, legolas_must_live = False, animate = False):
    
    grid = {(y,x):[v, 200] for y,row in enumerate(data.split('\n')) for x,v in enumerate(row)}

    if animate:
        print_grid(grid)
    else:
        clear_output()
        print(f"🧝‍♀️📶: {elf_hitpoints}\n")

    for round_num in range(0, 1000):
        game_over = False

        players_pos = sorted(pos for pos,(player,_) in grid.items() if player in 'GE')

        for cur_pos in players_pos:
            player = grid[cur_pos][0]
            if player == 'G':
                target = 'E'
            elif player == 'E':
                target = 'G'
            else:
                continue

            # MOVE
            if all(grid[p][0] != target for p in neighbours(*cur_pos)):
                dists = get_dists(grid, target = target)
                min_dist = min(dists.get(p, sys.maxsize) for p in neighbours(*cur_pos))

                if min_dist < sys.maxsize:
                    for dest_pos in neighbours(*cur_pos):
                        if dists.get(dest_pos, -1) == min_dist:
                            move(grid, cur_pos, dest_pos)
                            cur_pos = dest_pos
                            break

            targets = [(p,grid[p][1]) for p in neighbours(*cur_pos) if grid[p][0] == target]

            # ATTACK!
            if len(targets):
                target_pos = min(targets, key=lambda x:x[1])[0]
                if player == 'E':
                    hit(grid, target_pos, elf_hitpoints)
                else:
                    hit(grid, target_pos, 3)
                    if (grid[target_pos][1] <= 0) and legolas_must_live:
                        return False
                
                if animate:
                    print_grid(grid, attack_pos=target_pos,
                               elf_hitpoints=elf_hitpoints, round_num=round_num,
                               delay=0.05)
                    print_grid(grid, round_num=round_num, elf_hitpoints=elf_hitpoints)

            
        if len(set(p for p,_ in grid.values())) == 3:
            print_grid(grid, round_num=round_num, elf_hitpoints=elf_hitpoints)
            print("\nGAME OVER!")
            tot_points = sum(h for p,h in grid.values() if p in 'GE')
            print(round_num, 'x', tot_points, ' -> ', round_num*tot_points)
            return round_num*tot_points
                
        

In [169]:
res = simulate_combat(data, elf_hitpoints=3, legolas_must_live=False, animate=True)
print("Part 1:", res)

# for elf_hitpoints in range(4, 50):
#     res = simulate_combat(data, elf_hitpoints=elf_hitpoints, legolas_must_live=True, animate=False)
#     if res is not False:
#         break
# print("Part 2:", res)

Round #82
🧝‍♀️📶: 3

⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️▫️⬛️▫️⬛️▫️▫️▫️▫️▫️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️▫️▫️▫️▫️▫️▫️▫️▫️▫️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️▫️▫️▫️⬛️▫️⬛️▫️▫️▫️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️▫️⬛️⬛️▫️▫️⬛️▫️▫️▫️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️▫️▫️▫️▫️▫️▫️▫️▫️▫️▫️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️▫️▫️▫️▫️▫️▫️▫️▫️▫️▫️▫️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️▫️▫️▫️▫️▫️▫️▫️▫️▫️▫️▫️▫️▫️⬛️⬛️⬛️⬛️⬛️⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️▫️▫️⬛️▫️▫️▫️▫️▫️▫️▫️▫️▫️▫️⬛️⬛️⬛️⬛️⬛️⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️▫️⬛️⬛️▫️▫️▫️▫️▫️⬛️▫️▫️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️▫️▫️▫️▫️▫️▫️⬛️⬛️▫️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️⬛️▫️⬛️⬛️▫️▫️▫️▫️▫️▫️▫️▫️▫️▫️▫️▫️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️▫️▫️▫️▫️▫️▫️▫️▫️⬛️⬛️⬛️⬛️⬛️▫️👺▫️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️
⬛️⬛️⬛️⬛️⬛️▫️▫️▫️⬛️▫️▫️▫️▫️⬛️⬛️⬛️⬛️⬛️⬛️⬛️💀💀⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️
⬛️⬛️⬛️⬛️⬛️▫️▫️▫️▫️▫️👺▫️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️▫️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️
⬛️⬛️⬛️⬛️⬛